In [ ]:
# Combined_Script.py

# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML
from geopy.geocoders import Nominatim
import folium
from pptx import Presentation
from pptx.util import Inches, Pt


# Function for creating interactive graphs
def create_graphs():
    # Your graph creation code here
    # ... (include all the necessary imports and functions from your graph creation script)

    ## Basic import and DF creation
    import pandas as pd
    import plotly.graph_objects as go
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    from IPython.core.display import HTML

    # Specify the path to your Excel file
    file_path = r"03_Outputs\main_data.xlsx"

    # Load the Excel file into a DataFrame using the openpyxl engine
    df_main = pd.read_excel(file_path, engine='openpyxl')

    # Remove quotation marks from the 'Entity' column
    df_main['Entity'] = df_main['Entity'].str.replace('"', '')

    # Retrieve unique countries from the DataFrame and store them as a list of strings
    all_countries = df_main['Entity'].drop_duplicates().astype(str).tolist()

    # Display the first 5 rows of the DataFrame for a quick overview
    df_main.head()

    # Assuming df_main is already available
    filtered_df = df_main

    # Extract unique countries from the dataframe
    all_countries = sorted(filtered_df['Entity'].drop_duplicates().astype(str).tolist())
    default_countries = ['Germany', 'China', 'India', 'United States', 'World']
    left_container_width = '270px'

    # Custom sorting function
    def custom_sort(country):
        selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
        if country in selected_countries:
            return (-1, selected_countries.index(country))  # Selected countries come first
        else:
            return (1, country)  # All other countries come last

    # Update list of countries based on user search input
    def update_country_list(*args):
        selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
        search_input = search_box.value
        search_results = [country for country in all_countries if search_input.lower() in country.lower()]
        updated_countries = list(set(search_results + selected_countries))

        updated_countries = sorted(updated_countries, key=custom_sort)

        updated_checkboxes = [widgets.Checkbox(value=(country in selected_countries), description=country,
                                                layout=widgets.Layout(width="100%", padding="0px")) for country in updated_countries]
        checkboxes_container.children = tuple(updated_checkboxes)

        for cb in checkboxes_container.children:
            cb.observe(plot_selected_column, names='value')
            cb.observe(update_country_list, names='value')

    # The plot_selected_column function
    def plot_selected_column(change=None):
        column = column_dropdown.value
        selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
        with out:
            clear_output(wait=True)
            fig = go.Figure()
            for country in selected_countries:
                country_data = filtered_df[filtered_df['Entity'] == country]
                fig.add_trace(
                    go.Scatter(x=country_data['Year'], y=country_data[column], mode='lines', name=country,
                               showlegend=True))

            # Added dotted grid layout
            fig.update_layout(title=column,
                              xaxis=dict(title='Year', showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)',
                                         gridwidth=0.5, griddash='dot', zeroline=False),
                              yaxis=dict(title=column, showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)',
                                         gridwidth=0.5, griddash='dot', zeroline=False),
                              plot_bgcolor='white', paper_bgcolor='white')
            fig.show()

    checkboxes_container = widgets.VBox([], layout=widgets.Layout(overflow_y="scroll", height="400px",
                                                                  border="1px solid #DDD"))

    country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country,
                                          layout=widgets.Layout(width="100%", padding="0px")) for country in
                          sorted(all_countries, key=custom_sort)]

    for cb in country_checkboxes:
        cb.observe(plot_selected_column, names='value')

    checkboxes_container.children = tuple(country_checkboxes)

    # Creating the search box widget
    search_box = widgets.Text(value='', placeholder='Search for Countries...', description='',
                              layout=widgets.Layout(width="95%"))
    search_box.observe(update_country_list, names='value')

    clear_button = widgets.Button(description="X", layout=widgets.Layout(width="5%"))
    clear_button.on_click(lambda change: setattr(search_box, 'value', ''))

    out = widgets.Output(layout=widgets.Layout(height="400px", width="60%"))

    columns_to_plot = filtered_df.columns[filtered_df.columns.get_loc('Code') + 1:]
    column_dropdown = widgets.Dropdown(options=columns_to_plot, description='Select Column:')
    column_dropdown.observe(plot_selected_column, names='value')

    left_container = widgets.VBox([widgets.HBox([search_box, clear_button]), checkboxes_container],
                                  layout=widgets.Layout(width=left_container_width, border="1px solid #DDD",
                                                       padding="5px 0px 0px 0px"))

    display(HTML("""
    <style>
        :root {
            --jp-widgets-inline-label-width: auto !important;
        }
    </style>
    """))

    display(column_dropdown)
    display(widgets.HBox([left_container, out]))
    plot_selected_column()
    # Initial setup
    update_country_list()


# Function for creating maps
def create_map():
    # Your map creation code here
    import pandas as pd
    import plotly.express as px
    import ipywidgets as widgets
    from IPython.display import display

    # Data Import: Load Excel file into DataFrame
    file_path = "03_Outputs/main_data.xlsx"
    df_main = pd.read_excel(file_path)

    # Remove quotation marks from the 'Entity' column
    df_main['Entity'] = df_main['Entity'].str.replace('"', '')

    # Remove continents and 'World' to create choro_df (customize this list as needed)
    continents_and_world = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America', 'Antarctica',
                             'World']
    choro_df = df_main[~df_main['Entity'].isin(continents_and_world)]

    # Fixed reference columns
    fixed_columns = ['Entity', 'Year', 'Code']

    # List of columns to visualize (excluding the fixed columns)
    columns_to_visualize = [col for col in choro_df.columns if col not in fixed_columns]

    # Function to create interactive choropleth map
    def plot_choropleth(selected_column):
        df_filtered = choro_df[choro_df[selected_column].notna()]
        last_valid_year = df_filtered['Year'].max()

        # Get Top-5 countries for the last available year
        df_top5 = df_filtered[df_filtered['Year'] == last_valid_year]
        df_top5 = df_top5.nlargest(5, selected_column)

        print(f"Top 5 countries in {last_valid_year} for {selected_column}:")
        print(df_top5[['Entity', selected_column]])

        fig = px.choropleth(df_filtered.sort_values(by='Year', ascending=False),  # Sort data by Year in descending order
                            locations='Code',
                            color=selected_column,
                            hover_name='Entity',
                            animation_frame='Year',
                            title=f"Choropleth of {selected_column}",
                            color_continuous_scale=px.colors.sequential.Plasma)

        fig.update_layout(autosize=False, width=1000, height=600)
        fig.show()

    # Dropdown menu: Column selection for visualization
    dropdown = widgets.Dropdown(options=columns_to_visualize, description='Select Column:',
                               value=columns_to_visualize[0])

    # Interactive visualization
    interactive_plot = widgets.interactive(plot_choropleth, selected_column=dropdown)
    display(interactive_plot)

    
    # Function for creating PowerPoint presentations
def create_powerpoint():
    # Your PowerPoint creation code here
    from pptx import Presentation
    from pptx.util import Inches

# Create a new PowerPoint presentation
prs = Presentation()

def add_slide_with_text(prs, title, content):
    bullet_slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(bullet_slide_layout)
    shapes = slide.shapes
    title_shape = shapes.title
    body_shape = shapes.placeholders[1]

    title_shape.text = title
    title_shape.text_frame.text = title
    title_shape.text_frame.paragraphs[0].font.size = Pt(9)  # Set the font size to 16 points

      # Add a line break between title and images
    body_shape.text_frame.text = '\n'
    
    
    # Split the content into paragraphs and add them to the text frame
    for paragraph in content.split('\n'):
        p = body_shape.text_frame.add_paragraph()
        p.text = paragraph
        p.font.size = Pt(8)  # Set the font size to 12 points



# Create a new PowerPoint presentation
prs = Presentation()
        
# Add a title slide
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]  # Fixed typo here
title.text = "Renewable Energy Analysis \n Portfolio Project" 
subtitle.text = "Highlighting the importance of carbon dioxide reduction and the growth of the renewable energy sector \n From Data Analyst Group 2 _ Stackfuel Training School GmbH 2023"


# Define a function to add a slide with multiple images

def add_slide_with_images(prs, title, img_paths, image_width=9.0, image_height=8.0, space_above=0.8, title_font_size=None):    
    bullet_slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(bullet_slide_layout)
    shapes = slide.shapes
    title_shape = shapes.title
    body_shape = shapes.placeholders[1]
    
    
    # Clear the existing text in the title and body placeholders
    title_shape.text = ' '
    body_shape.text_frame.text = ' '

    title_shape.text = title
    tf = body_shape.text_frame

 # Adjust the margin_top only for Slide 7
    if prs.slides.index(slide) == 6:  # Slide index is 0-based, so Slide 7 has index 6
        tf.margin_top = Inches(space_above * 2)  # Add extra space for Slide 7
        if title_font_size:
            title_shape.text_frame.paragraphs[0].font.size = Pt(title_font_size / 2)  # Reduce the font size for Slide 7
    else:
        tf.margin_top = Inches(space_above)
        if title_font_size:
            title_shape.text_frame.paragraphs[0].font.size = Pt(title_font_size)
    
    
    # Define the positions and dimensions for the images
    left1, top1, width1, height1 = Inches(1), Inches(2), Inches(4), Inches(3)
    left2, top2, width2, height2 = Inches(5), Inches(2), Inches(4), Inches(3)

    # Add the first image if it exists
    if img_paths:
        pic1 = slide.shapes.add_picture(img_paths[0], left1, top1, width=width1, height=height1)

        # Resize the image
        pic1.left = left1
        pic1.top = top1
        pic1.width = width1
        pic1.height = height1

                        
    # Add the second image if it exists
    if len(img_paths) > 1:
        pic2 = slide.shapes.add_picture(img_paths[1], left=left2, top=top2, width=width2, height=height2)

        # Resize the image
        pic2.left = left2
        pic2.top = top2
        pic2.width = width2
        pic2.height = height2
        
# Add a slide with two images
add_slide_with_images(prs, "Annual Emission CO2 coal and flaring", ['Annual Emission CO2 coal.jpg', 'Annual Emission CO2 flaring.jpg'])

# Add another slide with two images
add_slide_with_images(prs, "Annual Emission CO2 Gas and Oil", ['Annual Emission CO2 gas .jpg', 'Annual Emission CO2 Oil.jpg'])

# Add another slide with two maps
add_slide_with_images(prs, "Total Annual of Emission in the World, 1965-2022", ['Total Annual CO2 Emission in the world 1965-2022.png', 'Primary Energy Consumption GDP (tons) in the world.png'])

# Add another slide with one map
add_slide_with_images(prs, "GDP in the World, 1965-2022", ['GDPWorld.png'])


# Add a slide with two images
add_slide_with_images(prs, "Electricity Production from Wind Energy and Solar Photovoltaic in tons", ['Electricity from Wind (Twh) .png', 'Solar Photovoltaic Capacity .png'])



# Add another slide with two images
add_slide_with_images(prs, "Others Renewable Energy excluding Bioenergy and Renewable (TWh Growth-equivalent) ", ['Other renewables energy excluding Bioenergy (TWh) .png', 'Renewables (TWh growth - equivalent)  .png'], title_font_size=5)


# Add another slide with one map
add_slide_with_images(prs, "Renewable Electricity Production in percent", ['Renewable Electricity Production (%).png'])


# Add a slide with the specified text at the end
add_slide_with_text(prs, "AI in Clean Energy Family Tree", """
We suggest that in future more AI machine learning projects in Clean Energy Family tree could solve Our Renewable energy problem.
Thermostat (AI device for home) – electricity usage can be monitored. Software, machine learning and AI come into play to help pave the way towards a low-emissions future.
A Deep Learning Model for Forecasting Photovoltaic Energy with Uncertainties – Computer Engineering, Virginia Commonwealth University.
Deep Reinforcement Learning for Optimal Energy Management of Multi-energy Smart Grids- Research University Bousnina D.
Tilting at Turbines could be the future of Wind Turbines.
Discovery of turning humidity (Humidity Batteries) into electricity which doesn’t have any emission could also change the future of Hydro plants.
Wave power with AI machines is also in the research phase which also adds up to emission free power production.
""")



# Save the presentation
prs.save('my_presentation_portfolio9.pptx')


        
# Main function to execute all tasks
def main():
    create_graphs()
    create_map()
    create_powerpoint()


# Run the main function if this script is executed
if __name__ == "__main__":
    main()
    
import os
os.system("my_presentation_portfolio9.pptx")
